In [1]:
#import necessary libraries
import pandas as pd 
import os
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

C:\Users\sayus\miniconda3\envs\te\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\sayus\miniconda3\envs\te\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\sayus\miniconda3\envs\te\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\sayus\miniconda3\envs\te\lib\site-packages\tensorflow\python\framework

In [2]:
#load test data
x_test = []
test_attributes = pd.read_csv('./split_data/test_attributes.csv')

In [3]:
#load test images to array
for index in test_attributes.index:
    img_name = test_attributes['filename'][index]
    img_ = cv.imread(os.path.join('./split_data/test', img_name + '.jpg'),1)
    x_test.append(img_)

In [4]:
y_test = test_attributes.drop('filename', axis = 1).values

In [5]:
x_test = np.array(x_test)
y_test = np.array(y_test)

In [6]:
x_test.shape, y_test.shape

((100, 300, 225, 3), (100, 3))

In [7]:
#preprocess input
x_test = tf.keras.applications.inception_v3.preprocess_input(x_test)

In [8]:
#convert categorical classes to one hot encoded vector
y_categorical_neck = to_categorical(y_test[:, 0], num_classes = 7)
y_categorical_sleeve_length = to_categorical(y_test[:,1], num_classes = 4)
y_categorical_pattern = to_categorical(y_test[:,2], num_classes = 10)

In [9]:
#concatenate one hote encoded vectors
y_test_final = np.append(y_categorical_neck, y_categorical_sleeve_length, axis = 1)

In [10]:
y_test_final = np.append(y_test_final, y_categorical_pattern, axis = 1)

In [11]:
y_test_final.shape

(100, 21)

In [12]:
from tensorflow.keras.models import load_model

In [13]:
#load the saved model with best accuracy
model = load_model('Best_Accuracy_Model.h5')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [14]:
#predict and evaluate
predictions = model.predict(x_test)
evaluation = model.evaluate(x_test, y_test_final)

100/100 [==============================] - 2s 15ms/sample - loss: 0.3055 - acc: 0.8824


In [15]:
predictions[:,0:7][0]

array([0.05229759, 0.0844996 , 0.01564622, 0.02732173, 0.10445148,
       0.03314269, 0.5448709 ], dtype=float32)

In [16]:
neck_predictions = predictions[:,0:7]
sleeve_length_predictions = predictions[:,7:11]
pattern_predictions = predictions[:,11:21]

In [17]:
neck_predictions.shape, y_categorical_neck.shape

((100, 7), (100, 7))

In [18]:
#convert to original form, by taking softmax function then reducing to categorical form to analyse predictions
neck_predictions = np.exp(neck_predictions)
sleeve_length_predictions = np.exp(sleeve_length_predictions)
pattern_predictions = np.exp(pattern_predictions)

In [19]:
neck_predictions_divide = neck_predictions.sum(axis = 1).reshape(100,1)
sleeve_length_predictions_divide = sleeve_length_predictions.sum(axis = 1).reshape(100,1)
pattern_prediction_divide = pattern_predictions.sum(axis = 1).reshape(100,1)

In [20]:
neck_prediction = neck_predictions/neck_predictions_divide
sleeve_length_predictions = sleeve_length_predictions/sleeve_length_predictions_divide
pattern_predictions = pattern_predictions/pattern_prediction_divide

In [21]:
#convert softmax probabilities to one hot encodings
neck_predictions = (neck_predictions == neck_predictions.max(axis=1)[:,None]).astype(int)
sleeve_length_predictions = (sleeve_length_predictions == sleeve_length_predictions.max(axis=1)[:,None]).astype(int)
pattern_predictions = (pattern_predictions == pattern_predictions.max(axis=1)[:,None]).astype(int)

In [22]:
from sklearn.metrics import confusion_matrix

In [23]:
#convert to categorical from one hot encoding
neck_predictions = neck_predictions.argmax(axis = 1)
sleeve_length_predictions = sleeve_length_predictions.argmax(axis = 1)
pattern_predictions = pattern_predictions.argmax(axis = 1)

In [24]:
neck_predictions.shape, y_test[:,0].shape

((100,), (100,))

In [25]:
#confusion matrix for neck attribute
confusion_matrix(y_test[:,0], neck_predictions)

array([[ 0,  0,  1,  0,  0,  0,  9],
       [ 0,  0,  1,  0,  0,  0, 10],
       [ 1,  0,  4,  2,  0,  0,  3],
       [ 0,  0,  7,  3,  0,  0,  3],
       [ 0,  0,  1,  1,  1,  0, 14],
       [ 0,  0,  5,  0,  0,  0,  4],
       [ 1,  1,  6,  1,  0,  0, 21]], dtype=int64)

In [26]:
#confusion matrix for sleeve_length attribute
confusion_matrix(y_test[:,1], sleeve_length_predictions)

array([[ 1,  0,  0,  9],
       [ 0,  0,  1, 17],
       [ 0,  0,  8,  3],
       [ 0,  3,  3, 55]], dtype=int64)

In [27]:
#confusion matrix for pattern attribute
confusion_matrix(y_test[:,2], pattern_predictions)

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  1,  0,  0,  0,  0,  0,  0, 16],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  5],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  3],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  4],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  1],
       [ 0,  0,  0,  1,  0,  0,  0,  0,  3],
       [ 0,  1,  0,  1,  1,  0,  0,  0, 60]], dtype=int64)

In [28]:
#create dataframe of final predictions
final_predictions = pd.DataFrame(columns = test_attributes.columns)

In [29]:
final_predictions['filename'] = test_attributes['filename']

In [30]:
final_predictions

,filename,neck,sleeve_length,pattern
0,11489747587291-Alcis-Men-Tshirts-6148974758719...,NaN,NaN,NaN
1,11489747633143-Alcis-Men-Tshirts-3791489747632...,NaN,NaN,NaN
2,11490005406481-Fort-Collins-Men-Red-Printed-Ma...,NaN,NaN,NaN
3,11490090151435-HRX-by-Hrithik-Roshan-Men-Black...,NaN,NaN,NaN
4,11490246959933-Louis-Philippe-Men-Cream-Printe...,NaN,NaN,NaN
...,...,...,...,...
95,11503394425849-LOCOMOTIVE-Men-Blue-Printed-Hen...,NaN,NaN,NaN
96,11503397290325-LOCOMOTIVE-Men-Blue-Printed-Rou...,NaN,NaN,NaN
97,11503484722306-American-Crew-Men-Grey-Striped-...,NaN,NaN,NaN
98,11503489114697-Adidas-Originals-Men-Peach-Colo...,NaN,NaN,NaN


In [31]:
#add predicted values to dataframe
final_predictions['neck'] = neck_predictions
final_predictions['sleeve_length'] = sleeve_length_predictions
final_predictions['pattern'] = pattern_predictions

In [32]:
final_predictions

,filename,neck,sleeve_length,pattern
0,11489747587291-Alcis-Men-Tshirts-6148974758719...,6,2,9
1,11489747633143-Alcis-Men-Tshirts-3791489747632...,6,2,9
2,11490005406481-Fort-Collins-Men-Red-Printed-Ma...,6,2,9
3,11490090151435-HRX-by-Hrithik-Roshan-Men-Black...,6,2,9
4,11490246959933-Louis-Philippe-Men-Cream-Printe...,1,3,4
...,...,...,...,...
95,11503394425849-LOCOMOTIVE-Men-Blue-Printed-Hen...,2,3,9
96,11503397290325-LOCOMOTIVE-Men-Blue-Printed-Rou...,6,3,9
97,11503484722306-American-Crew-Men-Grey-Striped-...,3,3,9
98,11503489114697-Adidas-Originals-Men-Peach-Colo...,2,3,9


In [33]:
#save the final predictions
final_predictions.to_csv('test_predictions.csv', index = False)